In [ ]:
import func
import pandas as pd
import re

In [ ]:
# 데이터 불러오기
raw = func.load_recipe(n=100) 
step = raw[["recipe_title","recipe_ingredients", "recipe_step"]].copy()

In [ ]:
# 레시피 스텝 쪼개기 + 레시피 스텝 길이 계산
data = []
step_length = []
for i in step['recipe_step'] :
    recipe_list = eval(i)  #리스트 형태의 문자열을 리스트로 변환
    cleaned_list = [re.sub(r'#\d+\s', '', item) for item in recipe_list] # #1, #2 제거
    data.append(cleaned_list)
    step_length.append(len(cleaned_list))
df = pd.DataFrame(data)

# 칼럼 이름 지어주기
max_step = df.shape[1]
col_name = [f"step{i}" for i in range(1, max_step+1)]
for i in range(max_step):
    df.rename(columns = {i : col_name[i]}, inplace= True)

# 레시피 제목, 레시피 식재료 목록, step 길이 더해주기
df["recipe_ingredients"] = step["recipe_ingredients"]
df["recipe_title"] = step['recipe_title']
df["step_length"] = step_length

In [ ]:
def recipe_preprocessing(raw):
    data = raw.loc[raw['recipe_ingredients'].notnull()].copy()  # None 값 제거
    def clean_ingredients(ingredients):
        if ingredients is not None:
            ingredients = ingredients.replace('\\ufeff', '').replace('\\u200b', '')
        return ingredients
    
    # recipe_ingredinents가 비어있지 않은 행만 남기기
    def not_empty_ingredients(row):
        return row['recipe_ingredients'].strip() != '{}' 

    data["recipe_ingredients"] = data["recipe_ingredients"].apply(clean_ingredients)
    result = data[data.apply(not_empty_ingredients, axis=1)].copy()

    title_idx = result[result['recipe_title'].isnull()].index # title이 null값인 행 인덱스 찾기
    del_idx = result[result['recipe_ingredients'].str.startswith('소시지')].index #소시지~ 로 시작해서 오류 일으키는 행 인덱스 찾기
    result.drop(del_idx, inplace=True) # 오류 일으키는 행 제거
    result.drop(title_idx, inplace=True) # title null값인 행 제거
    result = result.drop_duplicates() # 중복 제거

    return result

In [ ]:
import ast
from tqdm import tqdm

def find_ingredient(data):
    data = pd.concat([data, pd.DataFrame(columns= ["ingred_list"])])
    for idx, row in tqdm(data.iterrows(), total = data.shape[0]):
        ingred_list = []
        ingredients_dict = ast.literal_eval(row["recipe_ingredients"])
        for items in ingredients_dict.values():
            for item in items:
                match = re.match(r'([가-힣a-zA-Z]+(\([가-힣a-zA-Z]+\))?|\d+[가-힣a-zA-Z]*|\([가-힣a-zA-Z]+\)[가-힣a-zA-Z]+)([\d.+/~-]*)([가-힣a-zA-Z]+|약간|조금)?', item)
                if match :
                    ingredient, _, quantity, unit = match.groups()
                    ingred_list.append(ingredient)
                else : pass
        data.at[idx, "ingred_list"] = ingred_list
                
                

In [ ]:
df2 = pd.concat([df, pd.DataFrame(columns= ["ingred_list"])])
for idx, row in tqdm(df2.iterrows(), total = df2.shape[0]):
    ingred_list = []
    ingredients_dict = ast.literal_eval(row["recipe_ingredients"])
    for items in ingredients_dict.values():
        for item in items:
            match = re.match(r'([가-힣a-zA-Z]+(\([가-힣a-zA-Z]+\))?|\d+[가-힣a-zA-Z]*|\([가-힣a-zA-Z]+\)[가-힣a-zA-Z]+)([\d.+/~-]*)([가-힣a-zA-Z]+|약간|조금)?', item)
            if match :
                ingredient, _, quantity, unit = match.groups()
                ingred_list.append(ingredient)
            else : pass
    df2.at[idx, "ingred_list"] = ingred_list


In [78]:
df2.columns

Index(['step1', 'step2', 'step3', 'step4', 'step5', 'step6', 'step7', 'step8',
       'step9', 'step10', 'step11', 'step12', 'step13', 'recipe_ingredients',
       'recipe_title', 'step_length', 'ingred_list'],
      dtype='object')

In [85]:
df2.iloc[0].loc["step1"]

'먼저 둥근호박 하나를 반달썰기로 썰어 냄비에 넣고'

In [98]:
def find_ingredients(text, ingredient_list):
    for ingredient in ingredient_list:
        pattern = r"\b" + ingredient
        if re.search(pattern, text):
            print(ingredient)

ingredient_list = ["둥근호박", "청양고추", "당근약간", "실파약간", "새우젓", "물", "연두", "소금조금"]
find_ingredients("먼저 둥근호박 하나를 반달썰기로 썰어 냄비에 넣고 새우젓 냠냠 물을" , ingredient_list )

둥근호박
새우젓
물


In [103]:
df2.iloc[0].step1

'먼저 둥근호박 하나를 반달썰기로 썰어 냄비에 넣고'

In [ ]:
ingred_list

In [101]:
for idx, recipe in df2.iterrows():
    if idx < 10 : 
        print(idx)
        ingredients = recipe['ingred_list']
        for i in range(1, int(recipe.step_length) + 1):
            print("step" + f"{i}")
            print(recipe["step" + f"{i}"])
            print(find_ingredients(recipe["step" + f"{i}"], ingredients))


0
step1
먼저 둥근호박 하나를 반달썰기로 썰어 냄비에 넣고
둥근호박
None
step2
새우젓 1/2큰술, 물 100ml를 넣고 끓여요
새우젓
물
None
step3
센불에서 끓이다 호박이 익을무렵 약한불로 줄이고 미리 썰어놓은 청양고추 3개, 당근, 실파 그리고 입맛에 맞게 간을 맞추면 새우젓 호박찌개 완성
청양고추
새우젓
None
1
step1
된장 한 큰 술 넣어주었어요.
된장
None
step2
전복 듬뿍, 양파 1/2개, 둥근 호박, 감자 1개
전복
양파
감자
None
step3
우선 감자를 넣고 끓이다가 전복을 넣어끓여주었어요.다진 마늘도 넣고~~멸치 간 것 1스푼 넣어주었어요.10분 정도 끓여주었어요.
전복
감자
None
step4
대파도 송송 썰어 놓았지요.
대파
None
step5
10분정도 끓이다가 양파와 둥근 호박을넣어주었어요.
양파
None
step6
부들부들 두부도 작은 것 1/2모 넣어주었어요.
None
step7
송송 썰어 놓은 대파도 넣고 청양 고추를 넣고 고추가루도 넣고마무리해주었어요.정말 맛있는 전복 된장찌개가 완성되었어요.
된장
전복
대파
고추가루
None
2
3
step1
뚝배기에 양파,다진 마늘,바지락을 넣어 중불에서 살짝 볶는다.
바지락
양파
None
step2
1에 고춧가루를 넣어 볶다가 구수한 향이 나면 분량의 물을 넣고 순두부는 반으로 잘라 넣는다.
순두부
None
step3
2이 끓어 오르면 명품골드 다시다 쇠고기와 모든 양념재료를 넣고 끓인다. 중불로 줄이고 맛이 어우러지도록 끓인다.
None
step4
불을 끄고 계란을 넣어 완성한다.
None
4
step1
쇠고기는 2cm 크기로 네모나게 썰어 소금과 후추로 밑간한다.
None
step2
양파와 마늘은 곱게 다진다.
양파
마늘
None
step3
파프리카는 200℃ 오븐에 20분 정도 굽는다. 비닐백에 넣고 밀봉해 20분 정도 그대로 두었다가 꺼내 껍질을 벗겨낸 뒤 1.5cm 크기로 네모나게 썬다.
파프리카
None
step4
새송이버섯은 파프리카 크기로

In [ ]:
def find_ingredients(text):
    for ingredient in ingredient_list:
        pattern = r"\b" + ingredient + r"\b"
        if re.search(pattern, text):
            print(ingredient)

In [ ]:
result = find_ingredient(df)

In [ ]:


def split_ingredient_tiny(data):
    num_ingredients = 74

    # 식재료 이름, 양, 단위 칼럼 생성(모든 재료에 대한 열을 한 번에 생성)
    ingredient_columns = [f'ingredient{i}' for i in range(1, num_ingredients + 1)]
    quantity_columns = [f'quantity{i}' for i in range(1, num_ingredients + 1)]
    unit_columns = [f'unit{i}' for i in range(1, num_ingredients + 1)]

    # 새로운 DataFrame을 생성하여 모든 열을 한 번에 추가합니다.
    new_columns = ingredient_columns + quantity_columns + unit_columns
    data = pd.concat([data, pd.DataFrame(columns=new_columns)], axis=1)


    non_matching_items = {} # 패턴과 일치하지 않는 데이터를 저장할 딕셔너리

    for idx, row in tqdm(data.iterrows(), total=data.shape[0]): #tqdm으로 진행상황 확인
        if row['recipe_ingredients']:
            ingredients_dict = ast.literal_eval(row["recipe_ingredients"]) #딕셔너리 형태로 저장된 recipe_ingredients 불러오기
            ingredient_count = 1
            
            for items in ingredients_dict.values():
                if ingredient_count <= 75 : # 개별 레시피의 식재료 75개까지만
                    for item in items:
                        match = re.match(r'([가-힣a-zA-Z]+(\([가-힣a-zA-Z]+\))?|\d+[가-힣a-zA-Z]*|\([가-힣a-zA-Z]+\)[가-힣a-zA-Z]+)([\d.+/~-]*)([가-힣a-zA-Z]+|약간|조금)?', item)
                        if match:
                            ingredient, _, quantity, unit = match.groups()

                            data.at[idx, f'ingredient{ingredient_count}'] = ingredient
                            data.at[idx, f'quantity{ingredient_count}'] = quantity
                            data.at[idx, f'unit{ingredient_count}'] = unit

                            ingredient_count += 1
                        else:
                            non_matching_items[idx] = item
                else : pass

    data = data.drop([k for k, v in non_matching_items.items() if v != ''])

    return data

In [ ]:
data = recipe_preprocessing(df)
data_2 = split_ingredient_tiny(data)